<a href="https://colab.research.google.com/github/cheong0522/KBSC/blob/main/ing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
!pip install soynlp emoji

In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AdamW 
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
device = torch.device("cuda")

In [ ]:
test_data = pd.read_excel('/content/drive/MyDrive/KBSC_data.xlsx')

In [ ]:

test_data.loc[(test_data['Emotion'] == "행복"), 'Emotion'] = 0 
test_data.loc[(test_data['Emotion'] == "슬픔"), 'Emotion'] = 1  
test_data.loc[(test_data['Emotion'] == "분노"), 'Emotion'] = 2  
test_data.loc[(test_data['Emotion'] == "불안"), 'Emotion'] = 3  
test_data.loc[(test_data['Emotion'] == "중립"), 'Emotion'] = 4  

train_dataset = []
for sen, label in zip(test_data['Sentence'], test_data['Emotion']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))

  train_dataset.append(data_train)

class TrainDataset(Dataset):
  
  def __init__(self, dataset):
    self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

    self.sentences = [str([i[0]]) for i in dataset]
    self.labels = [np.int32(i[1]) for i in dataset]

  def __len__(self):
    return (len(self.labels))
  
  def __getitem__(self, i):
    text = self.sentences[i]
    y = self.labels[i]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

train_dataset = TrainDataset(train_dataset)


In [ ]:
from torch import nn